In [5]:
# 04_future_extensions_VAR_Markov.ipynb

# --- 1. Imports ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
from statsmodels.tsa.regime_switching.markov_switching import MarkovSwitching
from statsmodels.tsa.regime_switching.markov_autoregression import MarkovAutoregression

In [3]:
# --- 2. Load and Prepare Data ---
df = pd.read_csv("../data/brent_oil_prices.csv", parse_dates=["Date"])
df = df.set_index("Date")

In [6]:
# Ensure stationarity
df["Log_Returns"] = df["Price"].pct_change().apply(lambda x: np.log1p(x))
df = df.dropna()

In [11]:
# --- 3. Vector Autoregression (VAR) Example ---
np.random.seed(42)
df["USD_Index"] = df["Log_Returns"] + np.random.normal(0, 0.005, len(df))
df["Stock_Index"] = df["Log_Returns"].shift(1) + np.random.normal(0, 0.01, len(df))

model_data = df[["Log_Returns", "USD_Index", "Stock_Index"]].dropna()
var_model = VAR(model_data)
results = var_model.fit(maxlags=5, ic='aic')

print("✅ VAR Model Summary:")
print(results.summary())

✅ VAR Model Summary:
  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Tue, 05, Aug, 2025
Time:                     20:06:57
--------------------------------------------------------------------
No. of Equations:         3.00000    BIC:                   -27.1227
Nobs:                     9008.00    HQIC:                  -27.1289
Log likelihood:           83869.7    FPE:                1.64691e-12
AIC:                     -27.1321    Det(Omega_mle):     1.64472e-12
--------------------------------------------------------------------
Results for equation Log_Returns
                    coefficient       std. error           t-stat            prob
---------------------------------------------------------------------------------
const                  0.000189         0.000269            0.704           0.481
L1.Log_Returns         0.053211         0.054235            0.981           0.327
L1.USD_Index          -0.0675

C:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [10]:
# --- 4. Markov Regime Switching (on Log_Returns) ---
mod = MarkovAutoregression(df["Log_Returns"], k_regimes=2, order=1, switching_variance=True)
res = mod.fit()
print("✅ Markov Switching Summary:")
print(res.summary())

C:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


✅ Markov Switching Summary:
                         Markov Switching Model Results                         
Dep. Variable:              Log_Returns   No. Observations:                 9009
Model:             MarkovAutoregression   Log Likelihood               21912.675
Date:                  Tue, 05 Aug 2025   AIC                         -43809.350
Time:                          20:05:35   BIC                         -43752.502
Sample:                               0   HQIC                        -43790.007
                                 - 9009                                         
Covariance Type:                 approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0005      0.000      2.249      0.025    6.24e-05       0.001
sigma2  

In [12]:
# --- 5. Plot Regimes ---
df["Regime"] = res.smoothed_marginal_probabilities[1].idxmax(axis=1)
plt.figure(figsize=(12, 5))
plt.plot(df["Log_Returns"], label="Log Returns")
plt.plot(df["Regime"], label="Regime", alpha=0.6)
plt.title("Markov Regime Switching on Log Returns")
plt.legend()
plt.tight_layout()
plt.show()

ValueError: No axis named 1 for object type Series